In [174]:
import numpy as np
import cv2
from conf import SAMPLE_INPUTS,SAMPLE_OUTPUTS
from moviepy.editor import *
import pyautogui
import PIL
from PIL import Image,ImageGrab
import time
from pyautogui import screenshotUtil
from numpy import asarray
import natsort

In [137]:
all_frame_dir=os.path.join(SAMPLE_OUTPUTS,'all_frames')
output_vid=os.path.join(SAMPLE_OUTPUTS,'output.mp4')

In [138]:
img = cv2.imread('test_image.jpg')


In [139]:
def img_show(img):
    cv2.imshow('image',img)
    cv2.waitKey(600)
    cv2.destroyAllWindows()

### Canny edge detection

In [157]:
lane_img=np.copy(img)
gray=cv2.cvtColor((lane_img),cv2.COLOR_RGB2GRAY)# convert image ot gray scale

TypeError: Expected Ptr<cv::UMat> for argument 'src'

In [141]:
# reducing noise using gaussian blur
blur=cv2.GaussianBlur(gray,(5,5),0)
#img_show(blur)

In [142]:
canny=cv2.Canny(blur,50,150)  #50 is lower thresh and 150 is higher thresh
#img_show(canny)

### Region of interest

In [143]:
def region_of_interest(img):
    height=img.shape[0] 
    triangle=np.array([[(200,height),(1150,height),(550,250)]])   # list of triangle coordinates as our ROI
    mask=np.zeros_like(img)
    cv2.fillPoly(mask,triangle,255)
    masked_img=cv2.bitwise_and(img,mask)
    return masked_img

In [144]:
masked_img=region_of_interest(canny)
#img_show(region_of_interest(canny))

### Hough  Transform

In [145]:
def display_lines(img,lines):
    line_img=np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2=line.reshape(4)
            cv2.line(line_img,(x1,y1),(x2,y2),(0,255,0),10)
        return line_img
            

In [146]:
lines = cv2.HoughLinesP(masked_img,rho = 2,theta = 1*np.pi/180,threshold = 100,minLineLength = 60,maxLineGap = 5)   #hough lines for line detection and 2nd and 3rd arg are the bin size of x and y axis 4th arg is threshhold i.e min number of votes needed to accept a bin
print(lines)

[[[660 373 754 467]]

 [[747 472 927 658]]

 [[320 703 378 607]]

 [[900 613 976 689]]

 [[617 333 695 414]]

 [[592 304 704 416]]

 [[321 701 445 494]]

 [[704 418 927 641]]

 [[402 541 455 460]]

 [[704 426 766 490]]]


In [147]:
lines.shape

(10, 1, 4)

### Blending lines and original image


In [148]:
line_img=display_lines(lane_img,lines)
#img_show(line_img)

In [149]:
comb_img=cv2.bitwise_or(line_img,img)
#img_show(comb_img)

### Optimizing the lines

In [150]:
def make_coordinates_left(img,params):
    #slope,intercept=params
    try:
        slope, intercept = params
    except TypeError:
        slope, intercept =  (-1.63631916,1210.6787112)
    y1=img.shape[0]
    y2=420
    x1=int((y1-intercept)/slope)
    x2=int((x1+ (y2-y1)/slope))
    return np.array([x1,y1,x2,y2])  
   

In [151]:
def make_coordinates_right(img,params):
    #slope,intercept=params
    try:
        slope, intercept = params
    except TypeError:
        slope, intercept =  (0.99080842 -279.12458472)
    y1=img.shape[0]
    y2=420
    x1=int((y1-intercept)/slope)
    x2=int((x1+ (y2-y1)/slope))
    return np.array([x1,y1,x2,y2])  


In [152]:
def avg_slope_intercept(img,lines):
    left_fit=[]
    right_fit=[]
    for line in lines:
        x1,y1,x2,y2=line.reshape(4)
        x=np.array([x1,x2])
        y=np.array([y1,y2])
        param=np.polyfit(x,y,1)
        slope=param[0]
        intercept=param[1]
        if slope < 0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope,intercept))
    #if left_fit:
    left_fit_avg=np.average(left_fit,axis=0)
    
    left_line=make_coordinates_left(img,left_fit_avg)
    #if right_fit:
    right_fit_avg=np.average(right_fit,axis=0)    
    right_line=make_coordinates_right(img,right_fit_avg)
    print(right_fit_avg)
    time.sleep(100/1000)
    return np.array([left_line,right_line])

    

In [153]:
avg_lines=avg_slope_intercept(lane_img,lines)
print(avg_lines)

[   1.01486471 -293.76292095]
[[311 704 486 420]
 [983 704 703 420]]


In [154]:
line_img=display_lines(lane_img,avg_lines)

comb_img=cv2.bitwise_or(line_img,img)
img_show(comb_img)

### Detecting Lane in a video

In [155]:
cap=cv2.VideoCapture('test2.mp4')
i=0
while (cap.isOpened()):
    time.sleep(100/1000)
    _,frame=cap.read() 
    lane_img=np.copy(frame)
    gray=cv2.cvtColor(lane_img,cv2.COLOR_RGB2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0)
    canny=cv2.Canny(blur,50,150)
    masked_img=region_of_interest(canny)
    lines = cv2.HoughLinesP(masked_img,rho = 2,theta = 1*np.pi/180,threshold = 100,minLineLength = 60,maxLineGap = 5)   #hough lines for line detection and 2nd and 3rd arg are the bin size of x and y axis 4th arg is threshhold i.e min number of votes needed to accept a bin
    avg_lines=avg_slope_intercept(lane_img,lines)
    line_img=display_lines(lane_img,avg_lines)
    comb_img=cv2.bitwise_or(line_img,lane_img)
    image_save=os.path.join(all_frame_dir,f'{i}.jpg')
    img=Image.fromarray(comb_img).save(image_save)
    i=i+1


    

    

[   0.96595198 -267.90124546]
[   0.96492854 -266.75842132]
[   0.96447018 -266.31253886]
[   0.96447018 -266.31253886]
[   0.95927522 -263.01647486]
[   0.97936105 -276.24725388]
[   0.97936105 -276.24725388]
[   0.97403663 -274.25296043]
[   0.97403663 -274.25296043]
[   0.97075352 -266.96906207]
[   0.97075352 -266.96906207]
[   0.96852316 -266.19614805]
[   0.99080842 -279.12458472]
[   0.99080842 -279.12458472]
[   0.97974411 -271.63542952]
[   0.97974411 -271.63542952]
[   0.98689607 -277.25971667]
[   0.98689607 -277.25971667]
[   0.98077822 -271.67313703]
[   0.98077822 -271.67313703]
[   0.9828114  -271.96199143]
[   0.98410562 -272.79620592]
[   0.98801627 -276.16377923]
[   0.97792946 -269.15380625]
[   0.98650613 -274.08073603]
[   0.98000059 -269.28105288]
[   0.98000059 -269.28105288]
[   0.98590476 -272.51466667]
[   0.98788228 -270.9107635 ]
[   1.00502866 -283.22613832]
[   0.97693813 -264.72283392]
[   0.97693813 -264.72283392]
[   0.97890838 -266.45427974]
[   0.9885

[   1.01590524 -298.22258214]
[   1.01590524 -298.22258214]
[   1.0041312  -290.64020247]
[   1.0041312  -290.64020247]
[   1.00920175 -295.46353807]
[   1.00721892 -290.90353943]
[   1.01230661 -296.9323488 ]
[   1.01366301 -295.51624602]
[   1.01404327 -295.85919001]
[   1.02688048 -303.72375801]
[   1.02688048 -303.72375801]
[   1.0061082  -291.02879581]
[   1.00023432 -286.49855503]
[   1.02049528 -297.44243227]
[   1.02049528 -297.44243227]
[   1.00886239 -291.05655378]
[   1.01114665 -293.07444329]
[   1.01154776 -292.09977886]
[   1.02207045 -300.41723486]
[   1.02207045 -300.41723486]
[   1.01459033 -294.60859993]
[   1.01419138 -293.45488077]
[   1.00983269 -291.5310672 ]
[   1.00813631 -289.86876457]
[   1.01585433 -296.45731106]
[   1.00482708 -287.63465992]
[   1.01068946 -292.27171254]
[   0.9974817  -282.15258958]
[   1.01493876 -294.15380747]
[   1.00827689 -288.28364788]
[   1.00404057 -286.22346482]
[   1.00404057 -286.22346482]
[   1.01681569 -292.00805478]
[   1.0168

[   1.05468126 -313.43117765]
[   1.04835009 -308.75401451]
[   1.07351255 -327.49958837]
[   1.07351255 -327.49958837]
[   1.07351255 -327.49958837]
[   1.07351255 -327.49958837]
[   1.07351255 -327.49958837]
[   1.07351255 -327.49958837]
[   1.07288809 -326.84284828]
[   1.07288809 -326.84284828]
[   1.05692871 -315.24099002]
[   1.06778749 -322.23722761]
[   1.06778749 -322.23722761]
[   1.03994691 -302.48852742]
[   1.05935344 -317.91233617]
[   1.06630505 -322.9380389 ]
[   1.06630505 -322.9380389 ]
[   1.06619814 -323.63316343]
[   1.04914667 -311.14533654]
[   1.04914667 -311.14533654]
[   1.05844131 -317.35158688]
[   1.05844131 -317.35158688]
[   1.06616456 -322.65372077]
[   1.06616456 -322.65372077]
[   1.05319606 -314.04014464]
[   1.04477158 -308.53166564]
[   1.04477158 -308.53166564]
[   1.03391053 -300.18470418]
[   1.03391053 -300.18470418]
[   1.03551584 -302.35834224]
[   1.03713682 -303.72103734]
[   1.05035453 -311.94312287]
[   1.05035453 -311.94312287]
[   1.0600

[   1.11785882 -347.39883559]
[   1.11197885 -344.54001764]
[   1.10026513 -335.77880065]
[   1.10368704 -337.09711452]
[   1.10368704 -337.09711452]
[   1.10368704 -337.09711452]
[   1.10808328 -341.37539654]
[   1.10228243 -338.64622889]
[   1.11481943 -345.81170498]
[   1.11481943 -345.81170498]
[   1.11438575 -345.92316093]
[   1.11438575 -345.92316093]
[   1.10964839 -342.76746944]
[   1.12009325 -348.67989843]
[   1.12009325 -348.67989843]
[   1.10627852 -340.52086035]
[   1.10627852 -340.52086035]
[   1.107382   -339.03915799]
[   1.10457558 -338.54659222]
[   1.11030216 -342.24009412]
[   1.11030216 -342.24009412]
[   1.10202461 -336.88926089]
[   1.1084913  -342.01584967]
[   1.11882607 -348.8602405 ]
[   1.10606917 -339.92297734]
[   1.10606917 -339.92297734]
[   1.10997398 -342.47074536]
[   1.09611454 -334.15468516]
[   1.11332348 -346.2800076 ]
[   1.09447378 -332.38216473]
[   1.09212889 -331.45923476]
[   1.11855306 -350.06797816]
[   1.11855306 -350.06797816]
[   1.1028

[   1.06869908 -315.65223145]
[   1.05841254 -308.96785894]
[   1.06296214 -309.64827241]
[   1.05688567 -307.5109592 ]
[   1.06465795 -312.51824033]
[   1.0612993  -309.30745469]
[   1.04257794 -297.94005048]
[   1.06407906 -310.44564123]
[   1.06799114 -314.57805711]
[   1.06799114 -314.57805711]
[   1.07548832 -317.25409557]
[   1.07548832 -317.25409557]
[   1.07494142 -316.66280485]
[   1.06316006 -309.8377637 ]
[   1.06049864 -306.68570195]
[   1.06049864 -306.68570195]
[   1.08701118 -325.73636632]
[   1.07261504 -316.7491831 ]
[   1.07261504 -316.7491831 ]
[   1.07460431 -317.4897686 ]
[   1.06222672 -309.65033124]
[   1.05968971 -310.29102345]
[   1.05968971 -310.29102345]
[   1.05302978 -304.03484781]
[   1.07511043 -320.00991813]
[   1.06981684 -316.65757344]
[   1.059394  -308.7186221]
[   1.059394  -308.7186221]
[   1.07047685 -317.33536877]
[   1.06883101 -316.32463458]
[   1.06320372 -313.80015813]
[   1.0609592  -312.05456539]
[   1.06609873 -316.06872668]
[   1.07082012

TypeError: Expected Ptr<cv::UMat> for argument 'src'

In [178]:

inp_dir=os.listdir(all_frame_dir)

file_path=[] 
for fname in natsort.natsorted(inp_dir):
    if fname.endswith('jpg'):
        paths=os.path.join(all_frame_dir,fname)
        print(paths)
        file_path.append(paths)
output_clip=ImageSequenceClip(file_path,fps=20)
output_clip.write_videofile(output_vid)

/home/animesh/finding-lanes/data/samples/outputs/all_frames/0.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/1.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/2.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/3.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/4.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/5.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/6.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/7.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/8.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/9.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/10.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/11.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/12.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/13.jpg
/home/animesh/finding-lanes/data/samples/outputs/all_frames/14.jpg
/home

t:   1%|          | 9/1295 [00:00<00:14, 87.96it/s, now=None]

Moviepy - Building video /home/animesh/finding-lanes/data/samples/outputs/output.mp4.
Moviepy - Writing video /home/animesh/finding-lanes/data/samples/outputs/output.mp4



Moviepy - Done !
Moviepy - video ready /home/animesh/finding-lanes/data/samples/outputs/output.mp4
